In [0]:
from azure.storage.fileshare import ShareServiceClient
import os
import tqdm
from pathlib import Path
from pyspark.sql import functions as F


In [0]:
df = spark.sql("""
SELECT *
FROM 1_inland.sectra.pacs_file_copy
WHERE active_ind = 1 AND LOWER(status) != 'done'              
""")

In [0]:
display(df)

In [0]:
def concatSrcColsToPath(src_root, src_proj_dir, src_subdirs, src_filename):
    return F.concat(src_root, F.lit('/'), src_proj_dir, F.lit('/'), src_subdirs, F.lit('/'), src_filename)

In [0]:
def getSrcFileClient(src_path):
    acc_name = dbutils.secrets.get(scope = "adc_store", key = "pacs_intfileshare_accname")
    acc_key = dbutils.secrets.get(scope = "adc_store", key = "pacs_intfileshare_acckey")

    # Connection string
    connection_string = f"DefaultEndpointsProtocol=https;AccountName={acc_name};AccountKey={acc_key};EndpointSuffix=core.windows.net"

    # File share name
    share_name = "intfileshare"

    # Get a share client via connection string
    share_client = ShareServiceClient.from_connection_string(connection_string).get_share_client(share_name)

    file_client = share_client.get_file_client(src_path)

    return file_client

In [0]:
# TODO: make a UDF to move file from src to dst and return status

def copySrcFileToDst(row):
    src_path = os.path.join(row["src_root"], row["src_proj_dir"], row["src_subdirs"], row["src_filename"])

    #try:
    file_client = getSrcFileClient(src_path)
    file_bytes = file_client.download_file().readall()
    print(Path(row["dst_filepath"]).parent.absolute())

    # Make parent directory if not exist
    Path(row["dst_filepath"]).parent.absolute().mkdir(parents=True, exist_ok=True)

    # Write to Databricks
    with open(row["dst_filepath"], "wb") as f:
        f.write(file_bytes)
    #except Exception as e:
    #    print(e)
    #    raise e
    #    return (row["id"], 'failed')
    
    return (row["id"], 'done')


In [0]:
df2 = df.limit(2).withColumn("src_path", concatSrcColsToPath(F.col("src_root"), F.col("src_proj_dir"), F.col("src_subdirs"), F.col("src_filename"))).collect()



In [0]:
results = []
for row in df2:
    results.append(copySrcFileToDst(row))

print(results)